In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("./dataset/DM1_game_dataset.csv")

## 1. Deleting the numcomments

In [4]:
df = df.drop(labels = "NumComments", axis = 1)

## 2. Understanding commaxplaytime and mfgplaytime

In [5]:
df[df["MfgPlaytime"] == df["ComMaxPlaytime"]]["BGGId"].count()

np.int64(21925)

In [6]:
df["ComMaxPlaytime"].sort_values(ascending = False)[0:10]

4349     60000
19044    22500
1713     17280
15642    14400
8120     12000
18374    12000
8052     10000
2059      8640
14258     7920
3497      6000
Name: ComMaxPlaytime, dtype: int64

In [7]:
df["ComMaxPlaytime"].mean()

np.float64(90.51352337514253)

In [8]:
df["ComMaxPlaytime"].median()

np.float64(45.0)

In [9]:
# Decided to drop MfgPlaytime
df = df.drop(labels = "MfgPlaytime", axis = 1)

## 3. Correction of IsReimplementation 

In [10]:
# Fixing IsReimplementation based on NumImplementations
df.loc[(df["IsReimplementation"] == 0) & (df["NumImplementations"] > 0), "IsReimplementation"] = 1

In [11]:
df[(df["IsReimplementation"] == 0) & (df["NumImplementations"] > 0)]

,BGGId,Name,Description,YearPublished,GameWeight,ComWeight,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,...,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens,Rating


Weighted Mean

In [12]:
# weighted comweight
df["WeightedComWeight"] = df["ComWeight"] * df["NumWeightVotes"]